In [1]:
import pandas as pd
fulldata=pd.read_csv("/content/olid-training-v1.0.tsv",sep='\t')
columns = ['id','subtask_b','subtask_c']
fulldata.drop(columns, axis=1, inplace=True)

In [2]:
fulldata.head(5)

,tweet,subtask_a
0,@USER She should ask a few native Americans wh...,OFF
1,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF
2,Amazon is investigating Chinese employees who ...,NOT
3,"@USER Someone should'veTaken"" this piece of sh...",OFF
4,@USER @USER Obama wanted liberals &amp; illega...,NOT


In [3]:
train_texts= fulldata["tweet"]
train_labels = fulldata["subtask_a"]

In [4]:
test_texts=pd.read_csv("/content/testset-levela.tsv",sep='\t')
columns = ['id']
test_texts.drop(columns, axis=1, inplace=True)
test_labels = pd.read_csv('/content/labels-levela.csv', header=None)
test_labels = test_labels.iloc[:,1]
   

In [5]:
test_texts= test_texts["tweet"]

In [6]:
type(train_labels)

pandas.core.series.Series

In [7]:
fulldata.head(5)

,tweet,subtask_a
0,@USER She should ask a few native Americans wh...,OFF
1,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF
2,Amazon is investigating Chinese employees who ...,NOT
3,"@USER Someone should'veTaken"" this piece of sh...",OFF
4,@USER @USER Obama wanted liberals &amp; illega...,NOT


In [8]:
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
train_labels= label_encoder.fit_transform(train_labels)
test_labels= label_encoder.fit_transform(test_labels)

In [11]:
from sklearn.model_selection import train_test_split
train_textssur, val_textssur, train_labelssur, val_labelssur = train_test_split(train_texts, train_labels, test_size=.2)

In [12]:
type(train_textssur)

pandas.core.series.Series

In [13]:
!pip install transformers
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 15.4 MB/s 
     |████████████████████████████████| 182 kB 69.3 MB/s 
     |████████████████████████████████| 7.6 MB 61.0 MB/s 


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [14]:
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk import word_tokenize
from nltk.corpus import stopwords
stopwords1 = set(stopwords.words("english"))
# Preprocessing of tweets
def clean(listoftweets):
    
    listoffiltered=[]
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    for i in range(0,len(listoftweets)):
         
         
         listoftweets[i]=listoftweets[i].replace('@USER', '') #Remove mentions (@USER)
         
         listoftweets[i]=listoftweets[i].replace('URL', '')
         listoftweets[i]=listoftweets[i].replace('&', 'and')
         listoftweets[i]=listoftweets[i].replace('<', '')
         listoftweets[i]=listoftweets[i].replace('>', '')
         listoftweets[i]=listoftweets[i].replace('\d+', '')
         listoftweets[i] = re.sub("[^a-zA-Z]",  # Search for all non-letters
                          " ",          # Replace all non-letters with spaces
                          (listoftweets[i])) 
         listoftweets[i]=emoji_pattern.sub(r'', listoftweets[i])
         listoftweets[i]=listoftweets[i].translate(str.maketrans('', '', string.punctuation))
         listoftweets[i]=listoftweets[i].strip()
         listoftweets[i] = re.sub(' +', ' ', listoftweets[i])
         listoftweets[i]=listoftweets[i].lower()

    return listoftweets
        #  word_tokens = word_tokenize(listoftweets[i])
        #  filteredsen = [w for w in word_tokens if w not in stopwords1 and w != ' ']
        #  listoffiltered.append(filteredsen)
    # return listoffiltered
    

     


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [15]:
type(train_texts)

pandas.core.series.Series

In [17]:
import string
import pandas as pd
import numpy as np

# train_texts1=[]
# val_texts1 = []
# test_texts1 = []
# storing tweets in 1D list for each dataset for tokenization
listoftweetstrain_text = []
listoftweettest_text = []
listoftweetval_text = []
for i in range(0,len(train_textssur)):
      listoftweetstrain_text.append(train_textssur.iloc[i])
trainsur = clean(listoftweetstrain_text)
for i in range(0,len(val_textssur)):
      listoftweetval_text.append(val_textssur.iloc[i])
valsur = clean(listoftweetval_text)

for i in range(0,len(test_texts)):
      listoftweettest_text.append(test_texts.iloc[i])
testsur = clean(listoftweettest_text)


In [18]:
# Create encodings having words with vocab indices with attention mask encodings are in form of dictionary
train_encodings = (tokenizer(listoftweetstrain_text, truncation=True, padding=True))
val_encodings = (tokenizer(listoftweetval_text, truncation=True, padding=True))            
test_encodings = (tokenizer(listoftweettest_text, truncation=True, padding=True))

In [19]:
import torch
# Creation of dataset in pytorch
class Oliddataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Oliddataset(train_encodings, train_labelssur)
val_dataset = Oliddataset(val_encodings, val_labelssur)
test_dataset = Oliddataset(test_encodings, test_labels)

In [20]:
!pip install evaluate
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.4 MB/s 
     |████████████████████████████████| 451 kB 24.4 MB/s 
     |████████████████████████████████| 212 kB 27.7 MB/s 
     |████████████████████████████████| 132 kB 38.1 MB/s 
     |████████████████████████████████| 127 kB 58.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [21]:
# Compute metrics for metrics calculation like accuracy
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [23]:
## PYTORCH CODE
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=40,
)

model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,            # evaluation dataset
    compute_metrics=compute_metrics
)

trainer.train()


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_ve

Step,Training Loss
40,0.643000
80,0.556900
120,0.478500
160,0.453000




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=166, training_loss=0.5305619254169693, metrics={'train_runtime': 144.108, 'train_samples_per_second': 73.5, 'train_steps_per_second': 1.152, 'total_flos': 457221236451840.0, 'train_loss': 0.5305619254169693, 'epoch': 1.0})

In [25]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [26]:
predictions = trainer.predict(test_dataset)
print(predictions.predictions.shape, predictions.label_ids.shape)

***** Running Prediction *****
  Num examples = 860
  Batch size = 64


(860, 2) (860,)


In [30]:
predictions[:2]

(array([[-0.2160906 , -0.52261573],
        [ 0.13814867, -0.9087083 ],
        [ 0.09061909, -0.9583317 ],
        ...,
        [-0.71554613,  0.39765766],
        [-0.3748301 , -0.33602098],
        [ 0.71199614, -1.426597  ]], dtype=float32),
 array([1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1,
        0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0,
        0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
        1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
        1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
        0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
        0, 1, 

In [36]:
# Calculation of max probab and assigning the class label in accordance with that
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [37]:
preds.shape

(860,)

In [39]:
type(preds)

numpy.ndarray

In [41]:
df = pd.DataFrame(preds, columns = ['pred'])

# Remover column header and index
df.to_csv("predictionsfiletest.csv",index=False)


In [38]:
# Evaluation
import evaluate
metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8383720930232558, 'f1': 0.6759906759906759}